In [1]:
import random
import pandas as pd
from string import punctuation
from gensim.models.word2vec import Word2Vec
from config import Config
# from nlp.utils.pinyin import PinYin
from pypinyin import lazy_pinyin, pinyin

myconf = Config()
random.seed = myconf.random.seed
# pinyin = PinYin("./dict/word.data")
# pinyin.load_word()

In [2]:
def get_stopwords():
    with open("dict/stopwords.txt") as f:
        return [line.strip() for line in f.readlines()]

punctuation_zh = ".、，。°？！：；“”’‘～…【】『』［］（）()《》｛｝×―－·•→℃〈〉"
spec = ['"', ' ', '\xa0', '\n', '\ufeff', '\r', '\t']
# spec = ['\xa0', '\n', '\ufeff', '\r', '\t']
# Combination of Chinese and English punctuation.
punctuation_all = punctuation + punctuation_zh
# 表情
emoticon = myconf.data.emoticon
# 句尾语气词
tone_words = "的了呢吧吗啊啦呀么嘛哒哼"

# char level
# 1.不去
filter_characters = spec
# 2.去掉 标点+停用词+表情符号
# filter_characters = list(set(punctuation_all) | set(get_stopwords()) | set(emoticon)) + spec
# 3.去掉 标点+表情符号
# filter_characters = list(set(punctuation_all) | set(emoticon)) + spec
# 4.去掉 标点
# filter_characters = list(set(punctuation_all)) + spec

In [3]:
def seg_filter(text):
    """过滤分词。
    """
    # 句尾标点符号过滤
    s = text.rstrip(punctuation_all)
    # 句尾语气词过滤
    s = s.rstrip(tone_words)
    # 句中综合过滤
    words = [w for w in jieba.cut(s) if w not in filter_characters]
    return words

def seg_base(text):
    words = jieba.lcut(text, cut_all=False)
    return words
  
def filter_char(text, sep=' '):
    """Remove filter_characters and generate char sequence
    """
    res = [c for c in text if c not in filter_characters]
    return sep.join(res)

def get_chars(text):
    res = [c for c in text if c not in filter_characters]
    return res

def get_pinyin(text):
    res = [pinyin.char2pinyin(c) for c in text if c not in filter_characters]
    return res

def get_pinyin2(text, tone=False):
    res = filter_char(text, sep='')
    if tone:
        return lazy_pinyin(res, 1)
    else:
        return lazy_pinyin(res)

In [6]:
train = pd.read_csv(myconf.data.train_download)
train.content = train.content.map(lambda x: filter_char(x))
# train.content = train.content.map(lambda x: get_chars(x))
# train.content = train.content.map(lambda x: get_pinyin2(x, tone=True))
train["content"].head()

0    吼 吼 吼 ， 萌 死 人 的 棒 棒 糖 ， 中 了 大 众 点 评 的 霸 王 餐 ， ...
1    第 三 次 参 加 大 众 点 评 网 霸 王 餐 的 活 动 。 这 家 店 给 人 整 ...
2    4 人 同 行 点 了 1 0 个 小 吃 榴 莲 酥 榴 莲 味 道 不 足 松 软 奶 ...
3    之 前 评 价 了 莫 名 其 妙 被 删 果 断 继 续 差 评 ！ 换 了 菜 单 价 ...
4    出 乎 意 料 地 惊 艳 ， 椰 子 鸡 清 热 降 火 ， 美 容 养 颜 ， 大 大 ...
Name: content, dtype: object

In [27]:
train.to_csv(myconf.data.train, index=False)

In [7]:
val = pd.read_csv(myconf.data.val_download)
val.content = val.content.map(lambda x: filter_char(x))
# val.content = val.content.map(lambda x: get_chars(x))
# val.content = val.content.map(lambda x: get_pinyin2(x, tone=True))
val["content"].head()

0    哎 ， 想 当 年 来 佘 山 的 时 候 ， 啥 都 没 有 ， 三 品 香 算 镇 上 ...
1    趁 着 国 庆 节 ， 一 家 人 在 白 天 在 山 里 玩 耍 之 后 ， 晚 上 决 ...
2    这 家 店 是 我 目 前 吃 到 的 最 干 净 的 串 串 店 ， 目 前 看 到 最 ...
3    中 午 和 领 导 偷 偷 跑 出 来 开 小 灶 啦 ， 到 这 家 来 顿 料 理 ， ...
4    拖 了 很 久 很 久 才 来 补 的 点 评 ， 之 前 和 朋 友 一 块 儿 去 的 ...
Name: content, dtype: object

In [29]:
val.to_csv(myconf.data.val, index=False)

In [8]:
testa = pd.read_csv(myconf.data.testa_download)
testa.content = testa.content.map(lambda x: filter_char(x))
# testa.content = testa.content.map(lambda x: get_chars(x))
# testa.content = testa.content.map(lambda x: get_pinyin2(x, tone=True))
testa["content"].head()

0    我 想 说 他 们 家 的 优 惠 活 动 好 持 久 啊 ， 我 预 售 的 时 候 买 ...
1    终 于 开 到 心 心 念 念 的 L A B l o f t 。 第 一 次 来 就 随 ...
2    地 理 位 置 好 ， 交 通 方 便 ， 就 在 1 2 4 车 站 对 面 交 通 方 ...
3    运 气 很 好 ， 抽 中 了 大 众 点 评 的 霸 王 餐 。 这 家 主 题 餐 厅 ...
4    幸 运 随 点 评 团 体 验 霸 王 餐 ， 心 情 好 ~ 蜀 九 香 刚 进 驻 泉 ...
Name: content, dtype: object

In [31]:
testa.to_csv(myconf.data.testa, index=False)

In [9]:
testb = pd.read_csv(myconf.data.testb_download)
testb.content = testb.content.map(lambda x: filter_char(x))
# testb.content = testb.content.map(lambda x: get_chars(x))
# testb.content = testb.content.map(lambda x: get_pinyin2(x, tone=True))
testb["content"].head()

0    可 以 说 工 作 日 中 午 的 这 个 套 餐 着 实 是 特 别 的 实 惠 啊 ， ...
1    位 置 很 不 错 ， 老 板 眼 光 很 好 👍 位 于 地 铁 站 附 近 ， 周 围 ...
2    和 朋 友 小 聚 ， 选 择 了 山 上 下 . 「 龙 虾 沙 拉 」 朋 友 不 喜 ...
3    地 址 : 湖 滨 路 北 1 0 5 8 号 （ 国 税 局 南 侧 ） 东 来 顺 往 ...
4    元 旦 中 午 来 吃 饭 ， 七 个 大 人 一 个 小 孩 。 第 一 次 吃 东 北 ...
Name: content, dtype: object

In [6]:
testb.to_csv(myconf.data.testb, index=False)

In [3]:
def process_line(srcfile=None, desfile=None, func=None, mode='w'):
    """按行读取源文件进行自定义处理，将结果保存到目标文件。
    直到遇到空行或者到达文件末尾为止。
    """
    assert srcfile is not None, "srcfile can not be None"
    assert desfile is not None, "desfile can not be None"
    assert func is not None, "func can not be None"
    with open(desfile, mode, encoding='UTF-8') as des:
        with open(srcfile, 'r', encoding='UTF-8') as src:
            for line in iter(src.readline, ''):
                des.write(func(line))
    print("process_line complete!")

def build_seq(line):
    """Build seq for train data"""
    items = line.rstrip().split(',')
    tag = ' '.join(items[2:])
    return items[1] + '\t' + tag + '\n'

def build_seq_test(line):
    """Build seq for test data"""
    items = line.rstrip().split(',')
    return items[1] + '\n'

def build_text(line):
    """Build text for word2vec"""
    items = line.rstrip().split('\t')
    return items[0] + '\n'

In [4]:
# build data for seq2seq
# 使用 filter_char 处理后去除表头生成 myconf.data
process_line(srcfile=myconf.data.train, desfile="preprocess/train.tsv", func=build_seq)
process_line(srcfile=myconf.data.val, desfile="preprocess/val.tsv", func=build_seq)
process_line(srcfile=myconf.data.test, desfile="preprocess/test.tsv", func=build_seq_test)

process_line complete!
process_line complete!
process_line complete!


In [4]:
# build data for word2vec
# tsv 无表头
process_line(srcfile="preprocess/base/train.tsv", desfile="preprocess/base/train.txt", func=build_text, mode='w')
process_line(srcfile="preprocess/base/val.tsv", desfile="preprocess/base/dev.txt", func=build_text, mode='w')
process_line(srcfile="preprocess/base/test.tsv", desfile="preprocess/base/test.txt", func=build_text, mode='w')

process_line complete!
process_line complete!
process_line complete!


In [27]:
def read_file_lines(filepath, start=0, nmax=0):
    """按行读取文件，读取指定行数 nmax，当 nmax=0 时按行读取全部
    """
    fp = open(filepath, 'r', encoding='UTF-8')
    if start > 0:
        for i in range(start):
            fp.readline()
    n = 0
    while True:
        content = fp.readline()
        if content == '' or (n >= nmax and nmax != 0):
            break
        n += 1
        yield content      
    fp.close()

def cut_valid(filepath='./preprocessval.tsv', n=None, pos=None):
    assert n is not None, "n can not be None"
    assert pos is not None, "pos can not be None"
    with open('./preprocess/val_{}.tsv'.format(str(pos)), 'w', encoding='UTF-8') as f:
        for line in read_file_lines(filepath, start=0, nmax=pos):
            f.write(line)
    with open('./preprocess/val_{}.tsv'.format(str(n-pos)), 'w', encoding='UTF-8') as f:
        for line in read_file_lines(filepath, start=pos, nmax=n-pos):
            f.write(line)

cut_valid(filepath='./preprocess/val.tsv', n=15000, pos=10000)

In [10]:
# sentences = []
# with open("./preprocess/chars.txt", 'r', encoding='UTF-8') as src:
#     for line in iter(src.readline, ''):
#         sentences.append(line.rstrip().split())
        
frames = [train["content"], val["content"], testa["content"], testb["content"]]
content = pd.concat(frames)
with open("chars.txt", 'w', encoding='UTF-8') as f:
    for s in content:
        f.write(s + '\n')
# sentences = [s for s in content]

# word2vec_model = Word2Vec(
#     sentences,
#     size=100,
#     window=5,
#     min_count=1,
#     workers=4,
#     iter=15
#     )
print("done")

done


In [18]:
word2vec_model.wv.save_word2vec_format("embedding/pinyin_all_wv.vector", binary=True)
word2vec_model.save("embedding/pinyin_all.vector")

In [29]:
word2vec_model.wv.save_word2vec_format("embedding/chars_all_wv.vector", binary=True)
word2vec_model.save("embedding/chars_all.vector")